# Import Libraries

In [2]:
import mysql.connector

In [3]:
import pickle
import ruptures as rpt
import sklearn

from prettytable import PrettyTable

# Connect to MySQL Database

In [22]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='slow675',
    database='amd_project'
    )

In [23]:
print(mydb)

In [24]:
mycursor = mydb.cursor()

# Load Dataset

In [18]:
%run data_cleanse.ipynb

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished Loading


# Predict Labels

In [19]:
%run predict.ipynb

Finished Prediction


# Analyze Predicted Results

In [29]:
%run summarize.ipynb

# Insert to Database

In [168]:
sql = "INSERT INTO Patient (ID, Dateandtime, Milliseconds, X, Y, Z, HR, Label) \
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_all_p_sorted['ID'], 
               df_all_p_sorted['timestamp'], 
               df_all_p_sorted['Milliseconds'],
               df_all_p_sorted['x'], 
               df_all_p_sorted['y'], 
               df_all_p_sorted['z'], 
               df_all_p_sorted['HR'], 
               df_all_p_sorted['label']):
    
    mycursor.execute(sql, row)

mydb.commit()

In [169]:
sql = "INSERT INTO AllDaySummary (ID, Date, TimeFrom, TimeUntil,\
    DurationSit, DurationSleep, DurationStand, DurationWalk,\
    CountSit, CountSleep, CountStand, CountWalk,\
    CountActive, CountInactive,\
    CountTotalActiveness, CountTransition, DurationPerTransition)\
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,\
    %s, %s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_summary_all['ID'], 
               df_summary_all['date'], 
               df_summary_all['from'],
               df_summary_all['to'], 
               df_summary_all['sit'], 
               df_summary_all['sleep'], 
               df_summary_all['stand'],
               df_summary_all['walk'],
               df_summary_all['sit count'], 
               df_summary_all['sleep count'], 
               df_summary_all['stand count'], 
               df_summary_all['walk count'], 
               df_summary_all['inactive count'], 
               df_summary_all['active count'], 
               df_summary_all['total count'], 
               df_summary_all['transition count'], 
               df_summary_all['duration per action']):
    
    mycursor.execute(sql, row)
    
mydb.commit()

In [32]:
sql = "INSERT INTO ActivityPeriod (ID, Date, TimeFrom, TimeUntil, Label)\
    VALUES (%s, %s, %s, %s, %s)"

for row in zip(df_act_period['ID'], 
               df_act_period['date'], 
               df_act_period['from'],
               df_act_period['to'],
               df_act_period['label']):
    
    mycursor.execute(sql, row)

mydb.commit()

# Show Results

In [170]:
mycursor.execute("SELECT * FROM Patient")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(50):
    print(myresult[x])
    

16578
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '117', 0.2464, 0.4739, 0.5764, 68.2053, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '278', 0.2462, 0.4745, 0.5737, 68.2053, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '439', 0.246, 0.4739, 0.5745, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '956', 0.246, 0.4754, 0.5762, 68.2053, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '08', 0.2453, 0.47, 0.5752, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '243', 0.246, 0.475, 0.5718, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '402', 0.2462, 0.4719, 0.5703, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '6', 0.2468, 0.4725, 0.5754, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '76', 0.247, 0.4747, 0.5722, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '922', 0.2472, 0.4776, 0.5737, 67.3655, 'stand')
(2001

In [33]:
mycursor.execute("SELECT * FROM ActivityPeriod")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(len(myresult)):
    print(myresult[x])

505
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50328), datetime.timedelta(0, 50363), 'stand')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50363), datetime.timedelta(0, 50398), 'sit')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50398), datetime.timedelta(0, 50400), 'stand')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50400), datetime.timedelta(0, 50437), 'stand')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50437), datetime.timedelta(0, 50438), 'walk')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50438), datetime.timedelta(0, 50444), 'stand')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50444), datetime.timedelta(0, 50444), 'walk')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50444), datetime.timedelta(0, 50445), 'stand')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50445), datetime.timedelta(0, 50445), 'walk')
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50445), 

In [171]:
mycursor.execute("SELECT * FROM AllDaySummary")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(len(myresult)):
    print(myresult[x])

8
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 46800), datetime.timedelta(0, 50400), datetime.timedelta(0, 35), datetime.timedelta(0), datetime.timedelta(0, 36), datetime.timedelta(0), 1, 0, 2, 0, 1, 2, 3, 2, datetime.timedelta(0, 1200))
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50400), datetime.timedelta(0, 54000), datetime.timedelta(0, 67), datetime.timedelta(0, 70), datetime.timedelta(0, 243), datetime.timedelta(0, 184), 16, 6, 88, 91, 22, 179, 201, 32, datetime.timedelta(0, 17))
(3001, datetime.date(2019, 3, 5), datetime.timedelta(0, 36000), datetime.timedelta(0, 39600), datetime.timedelta(0, 115), datetime.timedelta(0, 69), datetime.timedelta(0, 182), datetime.timedelta(0, 64), 12, 9, 32, 19, 21, 51, 72, 31, datetime.timedelta(0, 50))
(3002, datetime.date(2019, 3, 5), datetime.timedelta(0, 43200), datetime.timedelta(0, 46800), datetime.timedelta(0, 66), datetime.timedelta(0, 70), datetime.timedelta(0, 168), datetime.timedelta(0, 30), 6, 4, 25, 20, 10, 45

In [31]:
# mycursor.execute("DROP TABLE AllDaySummary")
# mycursor.execute("DROP TABLE ActivityPeriod")
# mycursor.execute("DROP TABLE Patient")

# mycursor.execute("DROP DATABASE amd_project")